In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import sys
import platform
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# --- Core Parameters ---
STUDY_START_DATE = '2021-03-01'
study_start = pd.to_datetime(STUDY_START_DATE) # Define this globally
# Define the root directory for raw data input
INPUT_ROOT = 'data/raw/'
# Define the primary output directory for all processed data and analysis files
OUTPUT_DIR = 'data/processed/pipeline'

# Ensure the output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Environment initialized successfully.")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"\nStudy Period: {STUDY_START_DATE} onwards")
print(f"All processed files will be saved to: {OUTPUT_DIR}")

Environment initialized successfully.
Pandas version: 2.3.2
NumPy version: 1.26.4

Study Period: 2021-03-01 onwards
All processed files will be saved to: data/processed/pipeline


In [2]:
# Cell 2.1.1a: Optional - Fetch historical data from Socrata API and create raw consolidated file
# Note: This cell is optional. If API credentials are not provided, it will skip to using local CSV files
import os
import pandas as pd
from sodapy import Socrata
from dotenv import load_dotenv

print("="*80)
print("BUNKER DATA PIPELINE: STAGE 1 (OPTIONAL API FETCH)")
print("="*80)

try:
    # Attempt to load credentials from a .env file
    # Ensure 'socrata_api_credentials.env' is in the same directory or provide the correct path
    if os.path.exists('socrata_api_credentials.env'):
        load_dotenv(dotenv_path='socrata_api_credentials.env')
        app_token = os.getenv("SOCRATA_APP_TOKEN")
        username = os.getenv("SOCRATA_USERNAME")
        password = os.getenv("SOCRATA_PASSWORD")
    else:
        app_token, username, password = None, None, None

    if not all([app_token, username, password]):
        print("\nWARNING: Missing API credentials. Skipping Socrata API fetch.")
        # As a fallback for the rest of the script to run, create an empty api_df
        api_df = pd.DataFrame(columns=['date', 'vlsfo_price', 'mgo_price'])
        api_df['date'] = pd.to_datetime(api_df['date'])


    else:
        print("\nAPI credentials loaded. Fetching data from Socrata...")
        client = Socrata("agtransport.usda.gov", app_token, username=username, password=password)
        start_date = "2021-03-01T00:00:00.000"
        end_date = "2025-09-25T00:00:00.000"
        date_filter = f"day BETWEEN '{start_date}' AND '{end_date}'"
        
        # Fetch data from the API
        bunker_results = client.get("4v3x-mj86", where=date_filter, limit=20000)
        
        if bunker_results:
            api_df = pd.DataFrame.from_records(bunker_results)
            # Select and rename necessary columns
            api_df = api_df[['day', 'vlsfo_fuel_oil_imo_2020_grade_0_5', 'marine_gas_oil']].copy()
            api_df.rename(columns={'day': 'date', 'vlsfo_fuel_oil_imo_2020_grade_0_5': 'vlsfo_price', 'marine_gas_oil': 'mgo_price'}, inplace=True)
            # Convert date column to datetime objects
            api_df['date'] = pd.to_datetime(api_df['date'])
            print(f"API data fetching complete. Loaded {len(api_df)} records.")
        else:
            print("No data returned from API.")
            api_df = pd.DataFrame(columns=['date', 'vlsfo_price', 'mgo_price'])
            api_df['date'] = pd.to_datetime(api_df['date'])


    # --- This section now runs regardless of API success ---
    manual_files = {'vlsfo': 'shipandbunker_vlsfo_past_month.csv', 'mgo': 'shipandbunker_mgo_past_month.csv'}
    manual_dataframes = []
    
    # Create dummy CSV files for demonstration if they don't exist
    for filename in manual_files.values():
        filepath = os.path.join(INPUT_ROOT, 'bunker', filename)
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        if not os.path.exists(filepath):
            print(f"Creating dummy file: {filepath}")
            dummy_data = "ColA,ColB,Date,Price\nVal1,Val2,2025-10-01,700.50\nVal3,Val4,2025-10-02,705.75"
            with open(filepath, 'w') as f:
                f.write(dummy_data)

    print("\nLoading and processing manual CSV files...")
    for fuel, filename in manual_files.items():
        filepath = os.path.join(INPUT_ROOT, 'bunker', filename)
        if os.path.exists(filepath):
            # FIX: Changed header=None to header=0 to correctly handle the header row.
            df = pd.read_csv(filepath, header=0, usecols=[2, 3], names=['date', f'{fuel}_price'])
            manual_dataframes.append(df)
        else:
            print(f"WARNING: Manual file not found: {filepath}")

    if len(manual_dataframes) == 2:
        vlsfo_manual_df, mgo_manual_df = manual_dataframes
        vlsfo_manual_df['date'] = pd.to_datetime(vlsfo_manual_df['date'])
        mgo_manual_df['date'] = pd.to_datetime(mgo_manual_df['date'])
        manual_df = pd.merge(vlsfo_manual_df, mgo_manual_df, on='date', how='outer')

        # Combine API and manual data
        # This condition handles cases where the API call might have failed and returned an empty DataFrame
        if not api_df.empty:
            combined_df = pd.concat([api_df, manual_df[manual_df['date'] > api_df['date'].max()]], ignore_index=True)
        else:
            combined_df = manual_df

        # Clean and finalize the dataframe
        for col in ['vlsfo_price', 'mgo_price']:
            combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')
        
        combined_df.sort_values(by='date', inplace=True)
        combined_df.drop_duplicates(subset='date', keep='last', inplace=True)
        
        # Save the consolidated data
        output_dir = os.path.join(INPUT_ROOT, 'bunker')
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, "bunker_data.csv")
        combined_df.to_csv(output_path, index=False, date_format='%Y-%m-%d')
        print(f"\nProcessing complete. Consolidated raw data saved to '{output_path}'.")
    else:
        print("\nCould not process manual files. Skipping consolidation.")

except Exception as e:
    print(f"\nERROR: An unexpected error occurred in the pipeline: {e}. Proceeding with existing local data if possible.")


BUNKER DATA PIPELINE: STAGE 1 (OPTIONAL API FETCH)

API credentials loaded. Fetching data from Socrata...
API data fetching complete. Loaded 1189 records.

Loading and processing manual CSV files...

Processing complete. Consolidated raw data saved to 'data/raw/bunker\bunker_data.csv'.


In [3]:
# Cell 2.1.1b: Load the consolidated raw data and save the canonical version
print("="*80)
print("BUNKER DATA: LOADING AND CREATING CANONICAL FILE")
print("="*80)

bunker_file = os.path.join(INPUT_ROOT, 'bunker/bunker_data.csv')
print(f"\nLoading consolidated bunker data from: {bunker_file}")

bunker_df = pd.read_csv(bunker_file)
print(f"  Loaded {len(bunker_df)} rows, {len(bunker_df.columns)} columns")
print(f"  Columns: {list(bunker_df.columns)}")

date_col = None
for col in bunker_df.columns:
    if col.lower() == 'date':
        date_col = col
        break
if date_col is None: raise ValueError("No date column found")

bunker_df[date_col] = pd.to_datetime(bunker_df[date_col])
bunker_df = bunker_df.rename(columns={date_col: 'Date'})

print(f"\nBunker Data Summary:")
print(f"  Date Range: {bunker_df['Date'].min()} to {bunker_df['Date'].max()}")
print(f"  VLSFO Mean: ${bunker_df['vlsfo_price'].mean():.2f}/MT")
print(f"  MGO Mean: ${bunker_df['mgo_price'].mean():.2f}/MT")
print(f"  Missing Values: VLSFO={bunker_df['vlsfo_price'].isna().sum()}, MGO={bunker_df['mgo_price'].isna().sum()}")

bunker_df = bunker_df[bunker_df['Date'] >= study_start]
print(f"\nFiltered to study period ({STUDY_START_DATE} onwards): {len(bunker_df)} rows")

os.makedirs(OUTPUT_DIR, exist_ok=True)
bunker_output = os.path.join(OUTPUT_DIR, 'bunker_canonical.csv')
bunker_df.to_csv(bunker_output, index=False)
print(f"  Saved to: {bunker_output}")

BUNKER DATA: LOADING AND CREATING CANONICAL FILE

Loading consolidated bunker data from: data/raw/bunker/bunker_data.csv
  Loaded 1201 rows, 3 columns
  Columns: ['date', 'vlsfo_price', 'mgo_price']

Bunker Data Summary:
  Date Range: 2021-03-01 00:00:00 to 2025-10-10 00:00:00
  VLSFO Mean: $647.69/MT
  MGO Mean: $858.33/MT
  Missing Values: VLSFO=0, MGO=0

Filtered to study period (2021-03-01 onwards): 1201 rows
  Saved to: data/processed/pipeline\bunker_canonical.csv


In [4]:
print("="*80)
print("BALTIC EXCHANGE SPOT RATES & INDICES ACQUISITION")
print("="*80)

baltic_file = os.path.join(INPUT_ROOT, 'baltic_exchange/baltic_exchange_historic_data_010118_061025.csv')
print(f"\nLoading Baltic Exchange data from: {baltic_file}")

baltic_df = pd.read_csv(baltic_file)
print(f"  Successfully loaded {len(baltic_df)} rows")
print(f"  Total columns: {len(baltic_df.columns)}")

print("\nPre-cleaning Baltic Exchange data...")
initial_rows = len(baltic_df)
baltic_df_clean = baltic_df.dropna(subset=['P1A_82', 'P3A_82'], how='all')
rows_dropped = initial_rows - len(baltic_df_clean)
print(f"  Dropped {rows_dropped} rows where both P1A_82 and P3A_82 were null")
print(f"  Remaining rows: {len(baltic_df_clean)}")

date_col = None
for col in baltic_df_clean.columns:
    if col.lower().strip() == 'date':
        date_col = col
        break
if date_col is None: raise ValueError("No date column found")

baltic_df_clean[date_col] = pd.to_datetime(baltic_df_clean[date_col])
baltic_df_clean = baltic_df_clean.rename(columns={date_col: 'Date'})

baltic_spot = baltic_df_clean[baltic_df_clean['Date'] >= study_start].copy()
print(f"\nFiltered to study period: {len(baltic_spot)} rows")

print("\nTarget Variable Coverage:")
for target in ['P1A_82', 'P3A_82']:
    coverage = (baltic_spot[target].notna().sum() / len(baltic_spot)) * 100
    print(f"  {target}: {coverage:.2f}% ({baltic_spot[target].notna().sum()}/{len(baltic_spot)} observations)")
    print(f"    Mean: ${baltic_spot[target].mean():.2f}, Std: ${baltic_spot[target].std():.2f}")
    print(f"    Range: ${baltic_spot[target].min():.2f} - ${baltic_spot[target].max():.2f}")

baltic_output = os.path.join(OUTPUT_DIR, 'baltic_spot_canonical.csv')
baltic_spot.to_csv(baltic_output, index=False)
print(f"\nSaved to: {baltic_output}")
print(f"  Shape: {baltic_spot.shape[0]} rows x {baltic_spot.shape[1]} columns")

BALTIC EXCHANGE SPOT RATES & INDICES ACQUISITION

Loading Baltic Exchange data from: data/raw/baltic_exchange/baltic_exchange_historic_data_010118_061025.csv
  Successfully loaded 1202 rows
  Total columns: 46

Pre-cleaning Baltic Exchange data...
  Dropped 53 rows where both P1A_82 and P3A_82 were null
  Remaining rows: 1149

Filtered to study period: 1149 rows

Target Variable Coverage:
  P1A_82: 100.00% (1149/1149 observations)
    Mean: $17156.13, Std: $8110.55
    Range: $4458.00 - $45050.00
  P3A_82: 100.00% (1149/1149 observations)
    Mean: $16725.27, Std: $7525.47
    Range: $5434.00 - $40687.00

Saved to: data/processed/pipeline\baltic_spot_canonical.csv
  Shape: 1149 rows x 46 columns


In [5]:
print("="*80)
print("BALTIC FORWARD ASSESSMENTS (BFA) ACQUISITION")
print("="*80)

bfa_file = os.path.join(INPUT_ROOT, 'baltic_exchange/bfa_panamax_20190923_20251010.csv')
print(f"\nLoading BFA data from: {bfa_file}")

bfa_df = pd.read_csv(bfa_file)
print(f"  Loaded {len(bfa_df)} rows, {len(bfa_df.columns)} columns")
print(f"  Columns: {list(bfa_df.columns)}")

date_col = None
for col in bfa_df.columns:
    if 'date' in col.lower():
        date_col = col
        break
if date_col: 
    bfa_df[date_col] = pd.to_datetime(bfa_df[date_col])
    bfa_df = bfa_df.rename(columns={date_col: 'Date'})
    bfa_df = bfa_df[bfa_df['Date'] >= study_start]
    print(f"  Filtered to study period: {len(bfa_df)} rows")

if 'RouteIdentifier' in bfa_df.columns and 'RouteAverage' in bfa_df.columns:
    print("\nPivoting BFA data from long to wide format...")
    bfa_wide = bfa_df.pivot(index='Date', columns='RouteIdentifier', values='RouteAverage').reset_index()
    print(f"  Pivoted to wide format: {len(bfa_wide)} rows, {len(bfa_wide.columns)} columns")
else:
    bfa_wide = bfa_df

print("\nForward Curve Structure Analysis:")
p1_cols = [col for col in bfa_wide.columns if 'P1EA_82' in str(col)]
p3_cols = [col for col in bfa_wide.columns if 'P3EA_82' in str(col)]
print(f"  P1EA_82 (Atlantic) contracts: {len(p1_cols)}")
print(f"  P3EA_82 (Pacific) contracts: {len(p3_cols)}")
print(f"  Total forward contracts: {len(p1_cols) + len(p3_cols)}")

bfa_output = os.path.join(OUTPUT_DIR, 'bfa_wide_canonical.csv')
bfa_wide.to_csv(bfa_output, index=False)
print(f"\nSaved to: {bfa_output}")

BALTIC FORWARD ASSESSMENTS (BFA) ACQUISITION

Loading BFA data from: data/raw/baltic_exchange/bfa_panamax_20190923_20251010.csv
  Loaded 25022 rows, 5 columns
  Columns: ['GroupDesc', 'ArchiveDate', 'RouteIdentifier', 'RouteAverage', 'FFADescription']
  Filtered to study period: 25022 rows

Pivoting BFA data from long to wide format...
  Pivoted to wide format: 1165 rows, 23 columns

Forward Curve Structure Analysis:
  P1EA_82 (Atlantic) contracts: 11
  P3EA_82 (Pacific) contracts: 11
  Total forward contracts: 22

Saved to: data/processed/pipeline\bfa_wide_canonical.csv


In [6]:
def extract_series_metadata(file_path):
    """
    Extract series numbers, indicator names, and units from Clarksons Excel file.
    
    Returns:
        dict: Mapping of column index to {series_number, indicator, unit}
    """
    try:
        # Read rows 4-6 (indices 3-5) to get metadata
        df_meta = pd.read_excel(file_path, header=None, nrows=6)
        
        series_numbers = df_meta.iloc[3, :]  # Row 4: Series numbers
        indicator_names = df_meta.iloc[4, :]  # Row 5: Indicator names
        units = df_meta.iloc[5, :]           # Row 6: Units/Date
        
        metadata = {}
        for col_idx in range(len(series_numbers)):
            # Skip first column (Date)
            if col_idx == 0:
                continue
                
            series_num = series_numbers.iloc[col_idx]
            indicator = indicator_names.iloc[col_idx]
            unit = units.iloc[col_idx]
            
            # Only include if series number exists
            if pd.notna(series_num):
                metadata[col_idx] = {
                    'series_number': str(int(series_num)) if not pd.isna(series_num) else 'Unknown',
                    'indicator': str(indicator) if pd.notna(indicator) else 'Unknown',
                    'unit': str(unit) if pd.notna(unit) else 'Unknown'
                }
        
        return metadata
    except Exception as e:
        return {}

def detect_frequency_from_format(date_series):
    """
    Detect frequency by examining the format of date values.
    """
    sample = date_series.dropna().head(10).astype(str)
    if len(sample) == 0: return 'Unknown'
    first_val = sample.iloc[0].strip()
    if 'Q' in first_val and '-' in first_val: return 'Quarterly'
    if len(first_val) == 4 and first_val.isdigit(): return 'Yearly'
    try:
        if '-' in first_val and len(sample) >= 2:
            date1 = pd.to_datetime(sample.iloc[0], format='%d-%m-%Y', errors='coerce')
            date2 = pd.to_datetime(sample.iloc[1], format='%d-%m-%Y', errors='coerce')
            if pd.notna(date1) and pd.notna(date2):
                diff = (date2 - date1).days
                if diff <= 3: return 'Daily'
                elif diff <= 10: return 'Weekly'
                elif diff <= 45: return 'Monthly'
    except:
        pass
    return 'Unknown'

def infer_frequency_from_dates(dates):
    """
    Infer frequency from parsed datetime objects.
    """
    clean_dates = dates.dropna().sort_values()
    if len(clean_dates) < 2: return 'Unknown'
    diffs = clean_dates.diff().dropna()
    median_diff = diffs.median()
    if median_diff <= pd.Timedelta(days=3): return 'Daily'
    elif median_diff <= pd.Timedelta(days=10): return 'Weekly'
    elif median_diff <= pd.Timedelta(days=45): return 'Monthly'
    elif median_diff <= pd.Timedelta(days=120): return 'Quarterly'
    else: return 'Yearly'

def parse_clarksons_dates(date_series, frequency):
    """
    Parse Clarksons dates based on detected frequency.
    """
    if frequency in ['Daily', 'Weekly', 'Monthly', 'Unknown']:
        try:
            parsed = pd.to_datetime(date_series, format='%d-%m-%Y', errors='coerce')
        except:
            parsed = pd.to_datetime(date_series, errors='coerce')
    elif frequency == 'Quarterly':
        def quarter_to_date(q_str):
            try:
                if pd.isna(q_str): return pd.NaT
                q_str = str(q_str).strip()
                if 'Q' in q_str and '-' in q_str:
                    quarter, year = q_str.split('-')
                    quarter_num = int(quarter.replace('Q', ''))
                    month = (quarter_num - 1) * 3 + 1
                    return pd.Timestamp(int(year), month, 1)
                return pd.NaT
            except:
                return pd.NaT
        parsed = date_series.apply(quarter_to_date)
    elif frequency == 'Yearly':
        def year_to_date(year_str):
            try:
                if pd.isna(year_str): return pd.NaT
                year = int(str(year_str).strip())
                return pd.Timestamp(year, 1, 1)
            except:
                return pd.NaT
        parsed = date_series.apply(year_to_date)
    else:
        parsed = pd.to_datetime(date_series, errors='coerce')
    return parsed

def construct_clarksons_headers(multi_index_cols):
    """
    Construct proper column headers from Clarksons multi-level structure.
    
    Structure (Excel rows 5-6 read as headers using header=[4, 5]):
    - First column: (indicator from row 5 or empty, 'Date' from row 6) -> 'Date'
    - Other columns: ('Indicator Name' from row 5, 'Unit' from row 6) -> 'Indicator Name_Unit'
    """
    new_cols = []
    for col in multi_index_cols:
        level0, level1 = str(col[0]), str(col[1])
        
        # Check if this is the Date column
        if level1.strip().lower() == 'date' or (pd.isna(col[0]) and 'date' in level1.lower()):
            new_cols.append('Date')
        else:
            # For other columns, combine indicator name with unit
            # Handle NaN or empty values in level0
            if pd.isna(col[0]) or level0.strip().lower() in ('nan', ''):
                # If level0 is empty, use only level1
                new_cols.append(level1.strip())
            else:
                # Combine indicator (level0) with unit (level1)
                combined = f"{level0.strip()}_{level1.strip()}"
                new_cols.append(combined.strip('_'))
    
    return new_cols

def get_column_signature(file_path):
    """
    Extract column signature for duplicate detection.
    Reads Excel rows 5-6 as headers (pandas header=[4, 5]).
    """
    try:
        df = pd.read_excel(file_path, header=[4, 5], nrows=1)
        if isinstance(df.columns, pd.MultiIndex):
            cols = construct_clarksons_headers(df.columns)
        else:
            cols = list(df.columns)
        return tuple(sorted(cols[1:]))  # Exclude Date column
    except:
        return tuple()

print("Clarksons helper functions defined successfully (WITH SERIES NUMBER EXTRACTION).")

Clarksons helper functions defined successfully (WITH SERIES NUMBER EXTRACTION).


In [7]:
print("="*80)
print("CLARKSONS SHIPPING INTELLIGENCE DATA ACQUISITION (WITH SERIES NUMBERS)")
print("="*80)

print("\nPhase 1: Building Clarksons file manifest...")
clarksons_root = os.path.join(INPUT_ROOT, 'clarksons/')
manifest = []
print(f"  Scanning directory: {clarksons_root}")
for root, dirs, files in os.walk(clarksons_root):
    for file in files:
        if file.endswith('.xlsx'):
            file_path = os.path.join(root, file)
            rel_path = os.path.relpath(file_path, clarksons_root)
            path_parts = rel_path.replace('\\', '/').split('/')
            economic_concept = '_'.join(path_parts[:-1]) if len(path_parts) > 1 else 'unknown'
            manifest.append({'file_path': file_path, 'economic_concept': economic_concept, 'frequency': 'Unknown'})

manifest_df = pd.DataFrame(manifest)
print(f"  Found {len(manifest_df)} Excel files")
print(f"  Unique economic concepts: {manifest_df['economic_concept'].nunique()}")

print("\nPhase 2: Selecting best file for each economic concept...")
frequency_priority = {'Daily': 5, 'Weekly': 4, 'Monthly': 3, 'Quarterly': 2, 'Yearly': 1, 'Unknown': 0}
selected_files = []
data_selection_log = []

for concept in manifest_df['economic_concept'].unique():
    concept_files = manifest_df[manifest_df['economic_concept'] == concept].copy()
    candidates = []
    for _, file_info in concept_files.iterrows():
        try:
            # Read Excel rows 5-6 as headers (indicator names + Date/units)
            df = pd.read_excel(file_info['file_path'], header=[4, 5])
            
            # Construct proper column headers
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = construct_clarksons_headers(df.columns)
            
            # Trim data at "Please note" footer
            date_col_data = df.iloc[:, 0].astype(str)
            end_idx = date_col_data[date_col_data.str.contains('Please note', case=False, na=False)].index
            if len(end_idx) > 0:
                df = df.iloc[:end_idx[0]]
            
            # Detect frequency and parse dates
            frequency = detect_frequency_from_format(df.iloc[:, 0])
            df['Date'] = parse_clarksons_dates(df.iloc[:, 0], frequency)
            if frequency == 'Unknown': 
                frequency = infer_frequency_from_dates(df['Date'])
            
            # Filter to study period
            df_study = df[df['Date'] >= study_start]
            if len(df_study) == 0: 
                continue
            
            # Check data completeness (exclude Date column)
            missing_pct = df_study.iloc[:, 1:].isna().mean().mean()
            if missing_pct >= 0.2: 
                continue
            
            # Store candidate information
            candidate_info = file_info.to_dict()
            candidate_info.update({
                'frequency': frequency,
                'freq_score': frequency_priority.get(frequency, 0),
                'col_signature': get_column_signature(file_info['file_path'])
            })
            candidates.append(candidate_info)
            data_selection_log.append({
                'concept': concept, 
                'file': os.path.basename(file_info['file_path']), 
                'selected': False
            })
        except Exception as e:
            # Silently skip files that cannot be processed
            pass

    # Select best candidate(s) for this concept
    if len(candidates) > 0:
        best_score = max(c['freq_score'] for c in candidates)
        best_freq_candidates = [c for c in candidates if c['freq_score'] == best_score]
        
        # Remove duplicates based on column signature
        unique_signatures = {}
        final_selected = []
        for candidate in best_freq_candidates:
            sig = candidate['col_signature']
            if sig not in unique_signatures:
                unique_signatures[sig] = candidate
                final_selected.append(candidate)
        
        selected_files.extend(final_selected)
        
        # Mark selected files in log
        selected_filenames = [os.path.basename(sf['file_path']) for sf in final_selected]
        for log in data_selection_log:
            if log['file'] in selected_filenames and log['concept'] == concept:
                log['selected'] = True

print(f"\n  Total files selected: {len(selected_files)}")

print("\nPhase 3: Parsing and consolidating selected Clarksons files (WITH SERIES NUMBERS)...")
all_clarksons_data = []
series_metadata_records = []

for file_info in selected_files:
    try:
        # Extract series metadata (row 4: series numbers)
        series_meta = extract_series_metadata(file_info['file_path'])
        
        # Read Excel rows 5-6 as headers (pandas header=[4, 5])
        df = pd.read_excel(file_info['file_path'], header=[4, 5])
        
        # Construct proper column headers
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = construct_clarksons_headers(df.columns)
        
        # Trim at "Please note" footer
        date_col_data = df.iloc[:, 0].astype(str)
        end_idx = date_col_data[date_col_data.str.contains('Please note', case=False, na=False)].index
        if len(end_idx) > 0:
            df = df.iloc[:end_idx[0]]
        
        # Parse dates
        df['Date'] = parse_clarksons_dates(df.iloc[:, 0], file_info['frequency'])
        
        # Filter to study period
        if file_info['frequency'] == 'Yearly': 
            df = df[df['Date'].dt.year >= study_start.year]
        else: 
            df = df[df['Date'] >= study_start]
        
        # Process each column with its series number
        for col_idx, col_name in enumerate(df.columns):
            if col_name == 'Date':
                continue
            
            # Get series metadata for this column
            # Note: col_idx in df may not match original Excel column index due to header processing
            # We need to map back to original column index
            original_col_idx = col_idx  # This needs adjustment based on how pandas read the file
            
            # Try to find metadata by matching column index
            series_number = 'Unknown'
            for meta_idx, meta in series_meta.items():
                if meta_idx == original_col_idx:
                    series_number = meta['series_number']
                    break
            
            # If not found by index, try to infer from metadata
            if series_number == 'Unknown' and len(series_meta) > 0:
                # Use a fallback: assign based on order
                meta_keys = sorted(series_meta.keys())
                if col_idx - 1 < len(meta_keys):  # -1 because Date is column 0
                    series_number = series_meta[meta_keys[col_idx - 1]]['series_number']
            
            # Create series name with economic concept prefix
            series_name = file_info['economic_concept'] + '_' + str(col_name)
            
            # Create long-format dataframe for this series
            series_df = pd.DataFrame({
                'Date': df['Date'],
                'SeriesNumber': series_number,
                'SeriesName': series_name,
                'Value': df[col_name]
            })
            
            # Remove null values
            series_df = series_df.dropna(subset=['Value'])
            
            if len(series_df) > 0:
                all_clarksons_data.append(series_df)
                
                # Record metadata
                series_metadata_records.append({
                    'SeriesNumber': series_number,
                    'SeriesName': series_name,
                    'EconomicConcept': file_info['economic_concept'],
                    'Indicator': col_name,
                    'Frequency': file_info['frequency'],
                    'SourceFile': os.path.basename(file_info['file_path'])
                })
        
    except Exception as e:
        print(f"    ERROR processing {os.path.basename(file_info['file_path'])}: {e}")

if all_clarksons_data:
    clarksons_canonical = pd.concat(all_clarksons_data, ignore_index=True)
    print(f"\nConsolidated Clarksons data: {len(clarksons_canonical)} rows")
    print(f"  Unique series: {clarksons_canonical['SeriesName'].nunique()}")
    print(f"  Unique series numbers: {clarksons_canonical['SeriesNumber'].nunique()}")
    
    # Save canonical dataset with series numbers
    clarksons_output = os.path.join(OUTPUT_DIR, 'clarksons_canonical.csv')
    clarksons_canonical.to_csv(clarksons_output, index=False)
    print(f"\nSaved to: {clarksons_output}")
    
    # Save series metadata
    series_metadata_df = pd.DataFrame(series_metadata_records)
    series_metadata_df = series_metadata_df.drop_duplicates(subset=['SeriesNumber', 'SeriesName'])
    metadata_output = os.path.join(OUTPUT_DIR, 'clarksons_series_metadata.csv')
    series_metadata_df.to_csv(metadata_output, index=False)
    print(f"\nSaved series metadata to: {metadata_output}")
    print(f"  Total series documented: {len(series_metadata_df)}")
else:
    print("\nWARNING: No Clarksons data was successfully processed")

CLARKSONS SHIPPING INTELLIGENCE DATA ACQUISITION (WITH SERIES NUMBERS)

Phase 1: Building Clarksons file manifest...
  Scanning directory: data/raw/clarksons/
  Found 258 Excel files
  Unique economic concepts: 77

Phase 2: Selecting best file for each economic concept...

  Total files selected: 70

Phase 3: Parsing and consolidating selected Clarksons files (WITH SERIES NUMBERS)...

Consolidated Clarksons data: 188161 rows
  Unique series: 634
  Unique series numbers: 632

Saved to: data/processed/pipeline\clarksons_canonical.csv

Saved series metadata to: data/processed/pipeline\clarksons_series_metadata.csv
  Total series documented: 634


In [8]:
print("="*80)
print("CLARKSONS SHIPPING INTELLIGENCE DATA ACQUISITION (WITH SERIES NUMBERS)")
print("="*80)

print("\nPhase 1: Building Clarksons file manifest...")
clarksons_root = os.path.join(INPUT_ROOT, 'clarksons/')
manifest = []
print(f"  Scanning directory: {clarksons_root}")
for root, dirs, files in os.walk(clarksons_root):
    for file in files:
        if file.endswith('.xlsx'):
            file_path = os.path.join(root, file)
            rel_path = os.path.relpath(file_path, clarksons_root)
            path_parts = rel_path.replace('\\', '/').split('/')
            economic_concept = '_'.join(path_parts[:-1]) if len(path_parts) > 1 else 'unknown'
            manifest.append({'file_path': file_path, 'economic_concept': economic_concept, 'frequency': 'Unknown'})

manifest_df = pd.DataFrame(manifest)
print(f"  Found {len(manifest_df)} Excel files")
print(f"  Unique economic concepts: {manifest_df['economic_concept'].nunique()}")

print("\nPhase 2: Selecting best file for each economic concept...")
frequency_priority = {'Daily': 5, 'Weekly': 4, 'Monthly': 3, 'Quarterly': 2, 'Yearly': 1, 'Unknown': 0}
selected_files = []
data_selection_log = []

for concept in manifest_df['economic_concept'].unique():
    concept_files = manifest_df[manifest_df['economic_concept'] == concept].copy()
    candidates = []
    for _, file_info in concept_files.iterrows():
        try:
            # Read Excel rows 5-6 as headers (indicator names + Date/units)
            df = pd.read_excel(file_info['file_path'], header=[4, 5])
            
            # Construct proper column headers
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = construct_clarksons_headers(df.columns)
            
            # Trim data at "Please note" footer
            date_col_data = df.iloc[:, 0].astype(str)
            end_idx = date_col_data[date_col_data.str.contains('Please note', case=False, na=False)].index
            if len(end_idx) > 0:
                df = df.iloc[:end_idx[0]]
            
            # Detect frequency and parse dates
            frequency = detect_frequency_from_format(df.iloc[:, 0])
            df['Date'] = parse_clarksons_dates(df.iloc[:, 0], frequency)
            if frequency == 'Unknown': 
                frequency = infer_frequency_from_dates(df['Date'])
            
            # Filter to study period
            df_study = df[df['Date'] >= study_start]
            if len(df_study) == 0: 
                continue
            
            # Check data completeness (exclude Date column)
            missing_pct = df_study.iloc[:, 1:].isna().mean().mean()
            if missing_pct >= 0.2: 
                continue
            
            # Store candidate information
            candidate_info = file_info.to_dict()
            candidate_info.update({
                'frequency': frequency,
                'freq_score': frequency_priority.get(frequency, 0),
                'col_signature': get_column_signature(file_info['file_path'])
            })
            candidates.append(candidate_info)
            data_selection_log.append({
                'concept': concept, 
                'file': os.path.basename(file_info['file_path']), 
                'selected': False
            })
        except Exception as e:
            # Silently skip files that cannot be processed
            pass

    # Select best candidate(s) for this concept
    if len(candidates) > 0:
        best_score = max(c['freq_score'] for c in candidates)
        best_freq_candidates = [c for c in candidates if c['freq_score'] == best_score]
        
        # Remove duplicates based on column signature
        unique_signatures = {}
        final_selected = []
        for candidate in best_freq_candidates:
            sig = candidate['col_signature']
            if sig not in unique_signatures:
                unique_signatures[sig] = candidate
                final_selected.append(candidate)
        
        selected_files.extend(final_selected)
        
        # Mark selected files in log
        selected_filenames = [os.path.basename(sf['file_path']) for sf in final_selected]
        for log in data_selection_log:
            if log['file'] in selected_filenames and log['concept'] == concept:
                log['selected'] = True

print(f"\n  Total files selected: {len(selected_files)}")

print("\nPhase 3: Parsing and consolidating selected Clarksons files (WITH SERIES NUMBERS)...")
all_clarksons_data = []
series_metadata_records = []

for file_info in selected_files:
    try:
        # Extract series metadata (row 4: series numbers)
        series_meta = extract_series_metadata(file_info['file_path'])
        
        # Read Excel rows 5-6 as headers (pandas header=[4, 5])
        df = pd.read_excel(file_info['file_path'], header=[4, 5])
        
        # Construct proper column headers
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = construct_clarksons_headers(df.columns)
        
        # Trim at "Please note" footer
        date_col_data = df.iloc[:, 0].astype(str)
        end_idx = date_col_data[date_col_data.str.contains('Please note', case=False, na=False)].index
        if len(end_idx) > 0:
            df = df.iloc[:end_idx[0]]
        
        # Parse dates
        df['Date'] = parse_clarksons_dates(df.iloc[:, 0], file_info['frequency'])
        
        # Filter to study period
        if file_info['frequency'] == 'Yearly': 
            df = df[df['Date'].dt.year >= study_start.year]
        else: 
            df = df[df['Date'] >= study_start]
        
        # Process each column with its series number
        for col_idx, col_name in enumerate(df.columns):
            if col_name == 'Date':
                continue
            
            # Get series metadata for this column
            # Note: col_idx in df may not match original Excel column index due to header processing
            # We need to map back to original column index
            original_col_idx = col_idx  # This needs adjustment based on how pandas read the file
            
            # Try to find metadata by matching column index
            series_number = 'Unknown'
            for meta_idx, meta in series_meta.items():
                if meta_idx == original_col_idx:
                    series_number = meta['series_number']
                    break
            
            # If not found by index, try to infer from metadata
            if series_number == 'Unknown' and len(series_meta) > 0:
                # Use a fallback: assign based on order
                meta_keys = sorted(series_meta.keys())
                if col_idx - 1 < len(meta_keys):  # -1 because Date is column 0
                    series_number = series_meta[meta_keys[col_idx - 1]]['series_number']
            
            # Create series name with economic concept prefix
            series_name = file_info['economic_concept'] + '_' + str(col_name)
            
            # Create long-format dataframe for this series
            series_df = pd.DataFrame({
                'Date': df['Date'],
                'SeriesNumber': series_number,
                'SeriesName': series_name,
                'Value': df[col_name]
            })
            
            # Remove null values
            series_df = series_df.dropna(subset=['Value'])
            
            if len(series_df) > 0:
                all_clarksons_data.append(series_df)
                
                # Record metadata
                series_metadata_records.append({
                    'SeriesNumber': series_number,
                    'SeriesName': series_name,
                    'EconomicConcept': file_info['economic_concept'],
                    'Indicator': col_name,
                    'Frequency': file_info['frequency'],
                    'SourceFile': os.path.basename(file_info['file_path'])
                })
        
    except Exception as e:
        print(f"    ERROR processing {os.path.basename(file_info['file_path'])}: {e}")

if all_clarksons_data:
    clarksons_canonical = pd.concat(all_clarksons_data, ignore_index=True)
    print(f"\nConsolidated Clarksons data: {len(clarksons_canonical)} rows")
    print(f"  Unique series: {clarksons_canonical['SeriesName'].nunique()}")
    print(f"  Unique series numbers: {clarksons_canonical['SeriesNumber'].nunique()}")
    
    # Save canonical dataset with series numbers
    clarksons_output = os.path.join(OUTPUT_DIR, 'clarksons_canonical.csv')
    clarksons_canonical.to_csv(clarksons_output, index=False)
    print(f"\nSaved to: {clarksons_output}")
    
    # Save series metadata
    series_metadata_df = pd.DataFrame(series_metadata_records)
    series_metadata_df = series_metadata_df.drop_duplicates(subset=['SeriesNumber', 'SeriesName'])
    metadata_output = os.path.join(OUTPUT_DIR, 'clarksons_series_metadata.csv')
    series_metadata_df.to_csv(metadata_output, index=False)
    print(f"\nSaved series metadata to: {metadata_output}")
    print(f"  Total series documented: {len(series_metadata_df)}")
else:
    print("\nWARNING: No Clarksons data was successfully processed")

CLARKSONS SHIPPING INTELLIGENCE DATA ACQUISITION (WITH SERIES NUMBERS)

Phase 1: Building Clarksons file manifest...
  Scanning directory: data/raw/clarksons/
  Found 258 Excel files
  Unique economic concepts: 77

Phase 2: Selecting best file for each economic concept...

  Total files selected: 70

Phase 3: Parsing and consolidating selected Clarksons files (WITH SERIES NUMBERS)...

Consolidated Clarksons data: 188161 rows
  Unique series: 634
  Unique series numbers: 632

Saved to: data/processed/pipeline\clarksons_canonical.csv

Saved series metadata to: data/processed/pipeline\clarksons_series_metadata.csv
  Total series documented: 634


In [9]:
# Load and examine the metadata
if os.path.exists(os.path.join(OUTPUT_DIR, 'clarksons_series_metadata.csv')):
    metadata = pd.read_csv(os.path.join(OUTPUT_DIR, 'clarksons_series_metadata.csv'))
    
    print("="*80)
    print("CLARKSONS SERIES METADATA VERIFICATION")
    print("="*80)
    
    print(f"\nTotal series documented: {len(metadata)}")
    print(f"Series with valid numbers (not 'Unknown'): {(metadata['SeriesNumber'] != 'Unknown').sum()}")
    print(f"Series with 'Unknown' numbers: {(metadata['SeriesNumber'] == 'Unknown').sum()}")
    
    print("\nFrequency distribution:")
    print(metadata['Frequency'].value_counts())
    
    print("\nSample records (first 10):")
    print(metadata[['SeriesNumber', 'Indicator', 'Frequency']].head(10).to_string(index=False))
    
    # Check for duplicate series numbers
    dup_series = metadata[metadata.duplicated(subset=['SeriesNumber'], keep=False)]
    if len(dup_series) > 0:
        print(f"\nWARNING: Found {len(dup_series)} records with duplicate series numbers")
        print("This may indicate series measured in different units/frequencies.")
    else:
        print("\nAll series numbers are unique (GOOD).")
else:
    print("Metadata file not found - series number extraction may have failed.")

CLARKSONS SERIES METADATA VERIFICATION

Total series documented: 634
Series with valid numbers (not 'Unknown'): 634
Series with 'Unknown' numbers: 0

Frequency distribution:
Frequency
Weekly     475
Monthly     91
Daily       68
Name: count, dtype: int64

Sample records (first 10):
 SeriesNumber                                                                  Indicator Frequency
        30235                                         Capesize Bulkcarrier Deliveries_No   Monthly
        30236                                        Capesize Bulkcarrier Deliveries_DWT   Monthly
       554898                          Average Capesize 2020s-built (Eco) Earnings_$/day    Weekly
       554906              Capesize 2020s-built (Bauxite) Kamsar - Yantai Earnings_$/day    Weekly
       554914        Capesize 2020s-built (Eco) (Coal) Baltimore - Kandla Earnings_$/day    Weekly
       554918     Capesize 2020s-built (Eco) (Coal) Baltimore - Rotterdam Earnings_$/day    Weekly
       554922       Cape

In [10]:
print("="*80)
print("MASTER CALENDAR CREATION")
print("="*80)

master_dates = baltic_spot['Date'].unique()
master_calendar = pd.DataFrame({'Date': sorted(master_dates)})

print(f"\nMaster Calendar created with {len(master_calendar)} unique business dates")
print(f"  Date range: {master_calendar['Date'].min()} to {master_calendar['Date'].max()}")

MASTER CALENDAR CREATION

Master Calendar created with 1149 unique business dates
  Date range: 2021-03-01 00:00:00 to 2025-10-06 00:00:00


In [11]:
print("\n" + "="*80)
print("DATA QUALITY ASSURANCE")
print("="*80)

bunker_canonical = pd.read_csv(os.path.join(OUTPUT_DIR, 'bunker_canonical.csv'), parse_dates=['Date'])
baltic_canonical = pd.read_csv(os.path.join(OUTPUT_DIR, 'baltic_spot_canonical.csv'), parse_dates=['Date'])
bfa_canonical = pd.read_csv(os.path.join(OUTPUT_DIR, 'bfa_wide_canonical.csv'), parse_dates=['Date'])
clarksons_canonical = pd.read_csv(os.path.join(OUTPUT_DIR, 'clarksons_canonical.csv'), parse_dates=['Date'])

print("\n1. Missing Value Analysis:")
for name, df in [("Bunker", bunker_canonical), ("Baltic", baltic_canonical), ("BFA", bfa_canonical)]:
    missing_pct = (df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100
    print(f"   {name}: {missing_pct:.2f}% missing values")

# Handle duplicate (Date, SeriesName) combinations before pivoting
# Keep last value for each duplicate (most recent data if multiple files have same series)
clarksons_canonical_deduplicated = clarksons_canonical.drop_duplicates(subset=['Date', 'SeriesName'], keep='last')
duplicates_removed = len(clarksons_canonical) - len(clarksons_canonical_deduplicated)
if duplicates_removed > 0:
    print(f"\n   Note: Removed {duplicates_removed} duplicate (Date, SeriesName) entries from Clarksons data")

clarksons_wide = clarksons_canonical_deduplicated.pivot(index='Date', columns='SeriesName', values='Value')
clarksons_missing = (clarksons_wide.isnull().sum() / len(clarksons_wide)) * 100
print(f"   Clarksons: {clarksons_missing.mean():.2f}% average missing (across {len(clarksons_missing)} series)")
print(f"              Max missing in any series: {clarksons_missing.max():.2f}%")

print("\n2. Duplicate Date Check:")
for name, df in [("Bunker", bunker_canonical), ("Baltic", baltic_canonical), ("BFA", bfa_canonical)]:
    duplicates = df['Date'].duplicated().sum()
    print(f"   {name}: {duplicates} duplicate dates")

print("\n3. Study Period Compliance (Start Date: 2021-03-01):")
for name, df in [("Bunker", bunker_canonical), ("Baltic", baltic_canonical), ("BFA", bfa_canonical), ("Clarksons", clarksons_canonical)]:
    min_date = df['Date'].min()
    compliant = "PASS" if min_date >= pd.Timestamp('2021-03-01') else "FAIL"
    print(f"   {name}: {min_date.strftime('%Y-%m-%d')} [{compliant}]")


DATA QUALITY ASSURANCE

1. Missing Value Analysis:
   Bunker: 0.00% missing values
   Baltic: 25.34% missing values
   BFA: 2.27% missing values
   Clarksons: 82.31% average missing (across 634 series)
              Max missing in any series: 97.97%

2. Duplicate Date Check:
   Bunker: 0 duplicate dates
   Baltic: 0 duplicate dates
   BFA: 0 duplicate dates

3. Study Period Compliance (Start Date: 2021-03-01):
   Bunker: 2021-03-01 [PASS]
   Baltic: 2021-03-01 [PASS]
   BFA: 2021-03-01 [PASS]
   Clarksons: 2021-03-01 [PASS]


In [12]:
summary_data = {
    'Dataset': ['Bunker Canonical', 'Baltic Spot Canonical', 'BFA Wide Canonical', 'Clarksons Canonical'],
    'Rows': [len(bunker_canonical), len(baltic_canonical), len(bfa_canonical), len(clarksons_canonical)],
    'Columns/Series': [
        bunker_canonical.shape[1]-1, 
        baltic_canonical.shape[1]-1, 
        bfa_canonical.shape[1]-1, 
        clarksons_canonical['SeriesName'].nunique()
    ],
    'Date Range Start': [
        bunker_canonical['Date'].min().strftime('%Y-%m-%d'),
        baltic_canonical['Date'].min().strftime('%Y-%m-%d'),
        bfa_canonical['Date'].min().strftime('%Y-%m-%d'),
        clarksons_canonical['Date'].min().strftime('%Y-%m-%d')
    ],
    'Date Range End': [
        bunker_canonical['Date'].max().strftime('%Y-%m-%d'),
        baltic_canonical['Date'].max().strftime('%Y-%m-%d'),
        bfa_canonical['Date'].max().strftime('%Y-%m-%d'),
        clarksons_canonical['Date'].max().strftime('%Y-%m-%d')
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*80)
print("CANONICAL DATASETS SUMMARY")
print("="*80)
print(summary_df.to_string(index=False))

print("\n" + "="*80)
print("DATA ACQUISITION COMPLETE")
print("All datasets loaded, processed, and verified successfully.")
print("="*80)


CANONICAL DATASETS SUMMARY
              Dataset   Rows  Columns/Series Date Range Start Date Range End
     Bunker Canonical   1201               2       2021-03-01     2025-10-10
Baltic Spot Canonical   1149              45       2021-03-01     2025-10-06
   BFA Wide Canonical   1165              22       2021-03-01     2025-10-10
  Clarksons Canonical 188161             634       2021-03-01     2025-10-03

DATA ACQUISITION COMPLETE
All datasets loaded, processed, and verified successfully.


In [13]:
print("="*80)
print("VIF-INFORMED CORRELATION ANALYSIS")
print("="*80)

CORR_THRESHOLD_HIGH = 0.95
CORR_THRESHOLD_MEDIUM = 0.85

print(f"\nCorrelation Thresholds:")
print(f"  High (drop one feature): r > {CORR_THRESHOLD_HIGH}")
print(f"  Medium (review): {CORR_THRESHOLD_MEDIUM} < r ≤ {CORR_THRESHOLD_HIGH}")

VIF-INFORMED CORRELATION ANALYSIS

Correlation Thresholds:
  High (drop one feature): r > 0.95
  Medium (review): 0.85 < r ≤ 0.95


In [14]:
print("\n[INFO] BFA Forward Curve Features - Term Structure Exemption")
print("=" * 80)

bfa_features = [col for col in bfa_canonical.columns if col != 'Date']
p1_bfa_features = [col for col in bfa_features if col.startswith('P1EA_82')]
p3_bfa_features = [col for col in bfa_features if col.startswith('P3EA_82')]

print(f"  P1EA_82 tenors: {len(p1_bfa_features)} contracts")
print(f"  P3EA_82 tenors: {len(p3_bfa_features)} contracts")
print(f"  Total BFA features: {len(bfa_features)}")
print("\n  ALL BFA features retained (term structure exemption).")


[INFO] BFA Forward Curve Features - Term Structure Exemption
  P1EA_82 tenors: 11 contracts
  P3EA_82 tenors: 11 contracts
  Total BFA features: 22

  ALL BFA features retained (term structure exemption).


In [15]:
print("\nAnalyzing Baltic Exchange feature correlations...")

baltic_features = [col for col in baltic_canonical.columns if col not in ['Date', 'P1A_82', 'P3A_82']]
baltic_feature_df = baltic_canonical[baltic_features].dropna(how='all', axis=1)
print(f"  Analyzing {len(baltic_feature_df.columns)} Baltic features (labels excluded)...")

baltic_corr = baltic_feature_df.corr()

def compare_null_counts(df, feature1, feature2):
    """Compare nulls and apply tie-breaker for Panamax-specific features."""
    null1 = df[feature1].isnull().sum()
    null2 = df[feature2].isnull().sum()
    if null1 < null2: return (feature1, feature2, null1, null2)
    if null2 < null1: return (feature2, feature1, null2, null1)
    
    # Tie-breaker logic
    f1_is_panamax = feature1.startswith('P') or feature1 == 'BPI'
    f2_is_panamax = feature2.startswith('P') or feature2 == 'BPI'
    if f1_is_panamax and not f2_is_panamax: return (feature1, feature2, null1, null2)
    if f2_is_panamax and not f1_is_panamax: return (feature2, feature1, null2, null1)
    
    return (feature1, feature2, null1, null2) # Default tie-breaker

baltic_high_corr_pairs = []
baltic_null_comparison_results = []
for i in range(len(baltic_corr.columns)):
    for j in range(i+1, len(baltic_corr.columns)):
        corr_val = abs(baltic_corr.iloc[i, j])
        if corr_val > CORR_THRESHOLD_MEDIUM:
            feature1 = baltic_corr.columns[i]
            feature2 = baltic_corr.columns[j]
            action = 'DROP_ONE' if corr_val > CORR_THRESHOLD_HIGH else 'REVIEW'
            if corr_val > CORR_THRESHOLD_HIGH:
                kept, dropped, nulls_kept, nulls_dropped = compare_null_counts(baltic_canonical, feature1, feature2)
                baltic_null_comparison_results.append({
                    'Feature_Kept': kept, 'Feature_Dropped': dropped, 'Correlation': corr_val,
                    'Nulls_Kept': nulls_kept, 'Nulls_Dropped': nulls_dropped
                })
            baltic_high_corr_pairs.append({'Feature_1': feature1, 'Feature_2': feature2, 'Correlation': corr_val, 'Action': action})

baltic_high_corr_df = pd.DataFrame(baltic_high_corr_pairs).sort_values('Correlation', ascending=False)
baltic_null_comparison_df = pd.DataFrame(baltic_null_comparison_results).sort_values('Correlation', ascending=False)

print(f"  Found {len(baltic_high_corr_df)} feature pairs with correlation > {CORR_THRESHOLD_MEDIUM}")
print(f"  Identified {len(baltic_null_comparison_df)} pairs with correlation > {CORR_THRESHOLD_HIGH} for VIF reduction.")


Analyzing Baltic Exchange feature correlations...
  Analyzing 43 Baltic features (labels excluded)...
  Found 148 feature pairs with correlation > 0.85
  Identified 71 pairs with correlation > 0.95 for VIF reduction.


In [16]:
# ==== BALTIC: Closed-form retained/dropped with reconciliation ====
print("="*80)
print("BALTIC: CLOSED-FORM FEATURE DECISIONS")
print("="*80)

# Universe (exclude Date and labels)
baltic_universe = [c for c in baltic_canonical.columns if c not in ['Date','P1A_82','P3A_82']]
expected_baltic = len(baltic_universe)  # should be 43

# Rebuild pair list with overlap counts (guard against spurious r)
pairs = []
X = baltic_canonical[baltic_universe]
cols = X.columns.tolist()
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        a, b = cols[i], cols[j]
        tmp = X[[a,b]].dropna()
        if len(tmp) < 10:
            continue
        r = tmp[a].corr(tmp[b])
        if pd.notna(r) and abs(r) > CORR_THRESHOLD_MEDIUM:
            pairs.append({'Feature_1': a, 'Feature_2': b,
                          'Correlation': abs(r), 'N_overlap': len(tmp),
                          'Action': 'DROP_ONE' if abs(r) > CORR_THRESHOLD_HIGH else 'REVIEW'})

baltic_pairs_df = pd.DataFrame(pairs).sort_values(['Correlation','N_overlap'], ascending=[False, False])
baltic_pairs_df.to_csv(os.path.join(OUTPUT_DIR, "correlation_baltic_pairs_full.csv"), index=False)

def tie_break_baltic(df, f1, f2):
    """
    Determine which feature to keep when both are highly correlated.
    Priority:
    1. Fewer null values
    2. Panamax-specific features (starts with 'P' or is 'BPI') when nulls are equal
    3. Alphabetical order as final fallback
    """
    n1, n2 = df[f1].isna().sum(), df[f2].isna().sum()
    
    # Primary criterion: data completeness
    if n1 < n2:
        return f1, f2  # Keep f1 (fewer nulls), drop f2
    if n2 < n1:
        return f2, f1  # Keep f2 (fewer nulls), drop f1
    
    # Secondary criterion: Panamax priority (only when nulls are equal)
    f1_is_panamax = f1.startswith('P') or f1 == 'BPI'
    f2_is_panamax = f2.startswith('P') or f2 == 'BPI'
    
    if f1_is_panamax and not f2_is_panamax:
        return f1, f2  # Keep Panamax feature f1
    if f2_is_panamax and not f1_is_panamax:
        return f2, f1  # Keep Panamax feature f2
    
    # Tertiary criterion: deterministic alphabetical fallback
    return (min(f1, f2), max(f1, f2))

to_drop, kept = set(), set()
for _, r in baltic_pairs_df[baltic_pairs_df['Action']=='DROP_ONE'].iterrows():
    a, b = r['Feature_1'], r['Feature_2']
    if a in to_drop or b in to_drop: 
        continue
    if a in kept and b not in kept: 
        to_drop.add(b); continue
    if b in kept and a not in kept: 
        to_drop.add(a); continue
    if a in kept and b in kept:
        k, d = tie_break_baltic(baltic_canonical, a, b)
        if d in kept: kept.remove(d)
        to_drop.add(d); continue
    k, d = tie_break_baltic(baltic_canonical, a, b)
    kept.add(k); to_drop.add(d)

baltic_dropped = sorted(to_drop)
baltic_retained = sorted(set(baltic_universe) - to_drop)

pd.DataFrame({'Feature': baltic_retained}).to_csv(os.path.join(OUTPUT_DIR,"features_baltic_retained.csv"), index=False)
pd.DataFrame({'Feature': baltic_dropped}).to_csv(os.path.join(OUTPUT_DIR,"features_baltic_dropped.csv"), index=False)

print(f"\nExpected (Baltic features excluding labels): {expected_baltic}")
print(f"Retained: {len(baltic_retained)} | Dropped_Correlation: {len(baltic_dropped)}")
print(f"Check sum: {len(baltic_retained) + len(baltic_dropped)} == {expected_baltic}")
print(f"\n[SAVED] Baltic pairs to: {os.path.join(OUTPUT_DIR, 'correlation_baltic_pairs_full.csv')}")
print(f"[SAVED] Baltic retained to: {os.path.join(OUTPUT_DIR, 'features_baltic_retained.csv')}")
print(f"[SAVED] Baltic dropped to: {os.path.join(OUTPUT_DIR, 'features_baltic_dropped.csv')}")

BALTIC: CLOSED-FORM FEATURE DECISIONS

Expected (Baltic features excluding labels): 43
Retained: 20 | Dropped_Correlation: 23
Check sum: 43 == 43

[SAVED] Baltic pairs to: data/processed/pipeline\correlation_baltic_pairs_full.csv
[SAVED] Baltic retained to: data/processed/pipeline\features_baltic_retained.csv
[SAVED] Baltic dropped to: data/processed/pipeline\features_baltic_dropped.csv


In [17]:
print("\nAnalyzing Clarksons feature correlations by category...")

def clean_clarksons_feature_name(feature_name):
    import re
    return re.sub(r'_Unnamed:.*$', '', feature_name)

def extract_category(series_name):
    parts = series_name.split('_')
    if len(parts) >= 3: return f"{parts[0]}_{parts[1]}_{parts[2]}"
    return series_name

series_inventory = clarksons_canonical.groupby('SeriesName').agg({'Date': ['min', 'max', 'count'], 'Value': lambda x: x.notna().sum()}).reset_index()
series_inventory.columns = ['SeriesName', 'First_Date', 'Last_Date', 'Total_Rows', 'Non_Null_Count']
series_inventory['SeriesName_Clean'] = series_inventory['SeriesName'].apply(clean_clarksons_feature_name)
series_inventory['Category'] = series_inventory['SeriesName'].apply(extract_category)

categories = series_inventory['Category'].unique()
print(f"  Analyzing {len(categories)} Clarksons categories...")
clarksons_correlation_summary = []

for category in categories:
    category_series = series_inventory[series_inventory['Category'] == category]['SeriesName'].tolist()
    if len(category_series) < 2: continue
    category_data = clarksons_canonical[clarksons_canonical['SeriesName'].isin(category_series)]
    category_wide = category_data.pivot(index='Date', columns='SeriesName', values='Value')
    valid_cols = category_wide.columns[category_wide.isnull().mean() < 0.5]
    category_wide = category_wide[valid_cols]
    if len(category_wide.columns) < 2: continue
    category_corr = category_wide.corr()
    high_corr_count = 0
    for i in range(len(category_corr.columns)):
        for j in range(i+1, len(category_corr.columns)):
            if abs(category_corr.iloc[i, j]) > CORR_THRESHOLD_HIGH: high_corr_count += 1
    clarksons_correlation_summary.append({
        'Category': category, 'Series_Count': len(category_wide.columns), 'High_Corr_Pairs': high_corr_count,
        'Redundancy_Rate': high_corr_count / (len(category_wide.columns) * (len(category_wide.columns) - 1) / 2) if len(category_wide.columns) > 1 else 0
    })

clarksons_summary_df = pd.DataFrame(clarksons_correlation_summary).sort_values('Redundancy_Rate', ascending=False)
print(f"  Category correlation analysis complete.")
print("\n  Top 5 categories by redundancy rate:")
if len(clarksons_summary_df) > 0:
    print(clarksons_summary_df.head(5).to_string(index=False))


Analyzing Clarksons feature correlations by category...
  Analyzing 20 Clarksons categories...
  Category correlation analysis complete.

  Top 5 categories by redundancy rate:
                     Category  Series_Count  High_Corr_Pairs  Redundancy_Rate
      port_congestion_turkish             3                3         1.000000
      economic_indicators_bfi             7               13         0.619048
economic_indicators_commodity             3                1         0.333333
         bulkers_data_panamax             9                7         0.194444
        bulkers_data_capesize            13               11         0.141026


In [18]:
# ==== CLARKSONS: Closed-form decisions with frequency-aware sparsity ====
print("="*80)
print("CLARKSONS: CLOSED-FORM SERIES DECISIONS")
print("="*80)

COMPLETENESS_CUTOFF = 0.50  # Require at least 50% of expected observations

# Build series inventory with frequency inference
series_inventory = clarksons_canonical.groupby('SeriesName').agg(
    First_Date=('Date','min'), 
    Last_Date=('Date','max'),
    Actual_Obs=('Date','count')
).reset_index()

series_inventory['Category'] = series_inventory['SeriesName'].apply(
    lambda s: '_'.join(s.split('_')[:3]) if len(s.split('_'))>=3 else s
)

expected_clx = len(series_inventory)  # should be 634

# Infer frequency for each series from actual observation dates
def infer_series_frequency(series_name):
    dates = clarksons_canonical[clarksons_canonical['SeriesName']==series_name]['Date'].sort_values()
    if len(dates) < 2:
        return 'Unknown'
    diffs = dates.diff().dropna()
    median_diff = diffs.median()
    
    if median_diff <= pd.Timedelta(days=3): 
        return 'Daily'
    elif median_diff <= pd.Timedelta(days=10): 
        return 'Weekly'
    elif median_diff <= pd.Timedelta(days=45): 
        return 'Monthly'
    elif median_diff <= pd.Timedelta(days=120): 
        return 'Quarterly'
    else: 
        return 'Yearly'

print("\nInferring frequency for each series...")
series_inventory['Frequency'] = series_inventory['SeriesName'].apply(infer_series_frequency)

# Calculate expected observations based on frequency and date range
def calc_expected_obs(first_date, last_date, frequency):
    days = (last_date - first_date).days
    if frequency == 'Daily': 
        return max(days, 1)
    elif frequency == 'Weekly': 
        return max(days // 7, 1)
    elif frequency == 'Monthly': 
        return max(days // 30, 1)
    elif frequency == 'Quarterly': 
        return max(days // 90, 1)
    elif frequency == 'Yearly': 
        return max(days // 365, 1)
    else: 
        return 1

series_inventory['Expected_Obs'] = series_inventory.apply(
    lambda row: calc_expected_obs(row['First_Date'], row['Last_Date'], row['Frequency']),
    axis=1
)

# Calculate completeness rate (actual vs expected for this frequency)
series_inventory['Completeness'] = series_inventory['Actual_Obs'] / series_inventory['Expected_Obs']
# Cap at 1.0 for cases where actual > expected (e.g., due to rounding)
series_inventory['Completeness'] = series_inventory['Completeness'].clip(upper=1.0)

print(f"  Frequency distribution:")
print(series_inventory['Frequency'].value_counts().sort_index())

# Make decision based on completeness (not absolute missing rate)
universe = series_inventory[['SeriesName','Category','Frequency','Completeness']].copy()
universe['Decision'] = np.where(
    universe['Completeness'] < COMPLETENESS_CUTOFF, 
    'Excluded_Sparsity', 
    'Candidate'
)

print(f"\n  Excluded due to low completeness (<{COMPLETENESS_CUTOFF}): {(universe['Decision']=='Excluded_Sparsity').sum()}")
print(f"  Candidates for correlation analysis: {(universe['Decision']=='Candidate').sum()}")

# Category-wise correlation only on Candidates
candidates = universe[universe['Decision']=='Candidate'].copy()
dropped_rows, kept_rows, pair_rows = [], [], []

# Build wide format only for correlation analysis (not for sparsity assessment)
clx_wide = clarksons_canonical.pivot(index='Date', columns='SeriesName', values='Value')

def resolve_category(cat, cand_df):
    ser_list = cand_df['SeriesName'].tolist()
    wide = clx_wide[ser_list]
    # Drop columns with all-null within this slice
    wide = wide.loc[:, wide.isnull().mean() < 1.0]
    if wide.shape[1] <= 1:
        for s in wide.columns:
            kept_rows.append({'SeriesName': s, 'Category': cat, 'Decision': 'Retained'})
        return

    # Build pairs
    pairs = []
    cols = list(wide.columns)
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            a, b = cols[i], cols[j]
            tmp = wide[[a,b]].dropna()
            if len(tmp) < 10:  # insufficient overlap
                continue
            r = tmp[a].corr(tmp[b])
            if pd.notna(r) and abs(r) > CORR_THRESHOLD_MEDIUM:
                pairs.append({'Category': cat, 'Series_1': a, 'Series_2': b,
                              'Correlation': abs(r), 'N_overlap': len(tmp),
                              'Action': 'DROP_ONE' if abs(r) > CORR_THRESHOLD_HIGH else 'REVIEW'})
    
    if pairs:
        dfp = pd.DataFrame(pairs).sort_values(['Correlation','N_overlap'], ascending=[False, False])
        pair_rows.append(dfp)
        # Greedy drop for > high threshold
        to_drop_cat, kept_cat = set(), set()
        
        def tie_break(a, b):
            # Use completeness instead of null count for tie-breaking
            comp_a = universe[universe['SeriesName']==a]['Completeness'].iloc[0]
            comp_b = universe[universe['SeriesName']==b]['Completeness'].iloc[0]
            if comp_a > comp_b: 
                return a, b
            if comp_b > comp_a: 
                return b, a
            return (min(a,b), max(a,b))
        
        for _, r in dfp[dfp['Action']=='DROP_ONE'].iterrows():
            a, b = r['Series_1'], r['Series_2']
            if a in to_drop_cat or b in to_drop_cat: 
                continue
            if a in kept_cat and b not in kept_cat: 
                to_drop_cat.add(b); continue
            if b in kept_cat and a not in kept_cat: 
                to_drop_cat.add(a); continue
            if a in kept_cat and b in kept_cat:
                k, d = tie_break(a,b)
                if d in kept_cat: kept_cat.remove(d)
                to_drop_cat.add(d); continue
            k, d = tie_break(a,b)
            kept_cat.add(k); to_drop_cat.add(d)
        
        final_keep = sorted(set(cols) - to_drop_cat)
        kept_rows.extend([{'SeriesName': s, 'Category': cat, 'Decision':'Retained'} for s in final_keep])
        dropped_rows.extend([{'SeriesName': s, 'Category': cat, 'Decision':'Dropped_Correlation'} for s in sorted(to_drop_cat)])
    else:
        # No high-corr pairs; retain all candidates in this category
        kept_rows.extend([{'SeriesName': s, 'Category': cat, 'Decision':'Retained'} for s in cols])

print("\n  Performing category-wise correlation analysis...")
for cat in candidates['Category'].unique():
    sub = candidates[candidates['Category']==cat]
    resolve_category(cat, sub)

# Save correlation pairs
clx_pairs_df = pd.concat(pair_rows, ignore_index=True) if len(pair_rows) else pd.DataFrame(
    columns=['Category','Series_1','Series_2','Correlation','N_overlap','Action']
)
clx_pairs_df.to_csv(os.path.join(OUTPUT_DIR,"correlation_clarksons_pairs_full.csv"), index=False)

clx_kept_df = pd.DataFrame(kept_rows)
clx_drop_df = pd.DataFrame(dropped_rows)

# Assemble final decisions (every one of the 634 gets a status)
final_decisions = pd.concat([
    universe[universe['Decision']=='Excluded_Sparsity'][['SeriesName','Category','Decision','Frequency','Completeness']],
    clx_kept_df.merge(universe[['SeriesName','Frequency','Completeness']], on='SeriesName', how='left'),
    clx_drop_df.merge(universe[['SeriesName','Frequency','Completeness']], on='SeriesName', how='left')
], ignore_index=True)

final_decisions = final_decisions[['SeriesName','Category','Decision','Frequency','Completeness']].drop_duplicates()

# Persist
final_path = os.path.join(OUTPUT_DIR, "features_clarksons_decisions.csv")
final_decisions.to_csv(final_path, index=False)

# Reconciliation audit
print(f"\n" + "="*80)
print(f"Expected (Clarksons series): {expected_clx}")
print(f"\nDecision Breakdown:")
print(final_decisions['Decision'].value_counts().rename('Count'))
print(f"\nCheck total: {len(final_decisions)} == {expected_clx}")
print(f"\n[SAVED] Clarksons pairs to: {os.path.join(OUTPUT_DIR, 'correlation_clarksons_pairs_full.csv')}")
print(f"[SAVED] Clarksons decisions to: {final_path}")

CLARKSONS: CLOSED-FORM SERIES DECISIONS

Inferring frequency for each series...
  Frequency distribution:
Frequency
Daily       68
Monthly     91
Weekly     475
Name: count, dtype: int64

  Excluded due to low completeness (<0.5): 0
  Candidates for correlation analysis: 634

  Performing category-wise correlation analysis...

Expected (Clarksons series): 634

Decision Breakdown:
Decision
Dropped_Correlation    492
Retained               142
Name: Count, dtype: int64

Check total: 634 == 634

[SAVED] Clarksons pairs to: data/processed/pipeline\correlation_clarksons_pairs_full.csv
[SAVED] Clarksons decisions to: data/processed/pipeline\features_clarksons_decisions.csv


In [19]:
print("\n" + "="*80)
print("SAVING CORRELATION ANALYSIS RESULTS")
print("="*80)

baltic_null_comparison_df.to_csv(os.path.join(OUTPUT_DIR, "correlation_baltic_redundancy_report.csv"), index=False)
print(f"[SAVED] Baltic redundancy report to: {os.path.join(OUTPUT_DIR, 'correlation_baltic_redundancy_report.csv')}")

clarksons_summary_df.to_csv(os.path.join(OUTPUT_DIR, "correlation_clarksons_category_summary.csv"), index=False)
print(f"[SAVED] Clarksons category summary to: {os.path.join(OUTPUT_DIR, 'correlation_clarksons_category_summary.csv')}")


SAVING CORRELATION ANALYSIS RESULTS
[SAVED] Baltic redundancy report to: data/processed/pipeline\correlation_baltic_redundancy_report.csv
[SAVED] Clarksons category summary to: data/processed/pipeline\correlation_clarksons_category_summary.csv


In [20]:
print("="*80)
print("COMPUTATIONAL ENVIRONMENT")
print("="*80)
print(f"Execution Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Python Version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"\nPackage Versions:")
print(f"  Pandas: {pd.__version__}")
print(f"  NumPy: {np.__version__}")
print("="*80)

print("\n" + "="*80)
print("END OF NOTEBOOK")
print("="*80)

COMPUTATIONAL ENVIRONMENT
Execution Date: 2025-10-18 16:36:27
Python Version: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]
Platform: Windows-11-10.0.26100-SP0

Package Versions:
  Pandas: 2.3.2
  NumPy: 1.26.4

END OF NOTEBOOK
